In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

## Generating Data

In [ ]:
seed = 6343
np.random.seed(seed)

In [ ]:
n = 15
m = 15

In [ ]:
farm_clusters = (np.random.random((n, 2)) * 10).round(decimals=1)
ranges = np.random.random((n, 2)) * 0.3

In [ ]:
ranges = ranges.round(decimals=1)

In [ ]:
farm_pixels = []
for i in range(n):
    xs = farm_clusters[i][0] - ranges[i][0]
    xe = farm_clusters[i][0] + ranges[i][0]
    
    ys = farm_clusters[i][1] - ranges[i][1]
    ye = farm_clusters[i][1] + ranges[i][1]
    
    for x in np.arange(xs, xe + 0.1, 0.1):
        for y in np.arange(ys, ye + 0.1, 0.1):
            farm_pixels.append([x, y])
    
farm_pixels = np.array(farm_pixels).round(decimals=1)

farm_pixels = farm_pixels[~((farm_pixels[:, 0] < 0) | (farm_pixels[:, 1] < 0))]

In [ ]:
wells = (np.random.random((m, 2)) * 10).round(decimals=1)

In [ ]:
plt.clf()
fig = plt.figure(figsize=(15, 15))
plt.scatter(farm_pixels[:, 0], farm_pixels[:, 1], marker="s", s=10, color="lightgreen")
plt.scatter(wells[:, 0], wells[:, 1], marker="s", s=15, color="blue")
plt.show()

In [ ]:
base_water_level = -np.random.randint(0, n)
distance_weight = np.random.random()
farm_factor = np.random.random()

In [ ]:
def get_observations(wells, farms, farm_factor, distance_weight, base):
    def pairwise_distances(x, y=None):
        x_norm = (x ** 2).sum(1)[:, None]
        y_norm = (y ** 2).sum(1)[None, :]
        
        dist = x_norm + y_norm - 2.0 * np.matmul(x, y.T)
        
        return dist
    
    obs = base - np.sum(np.exp(-pairwise_distances(wells, farms) / distance_weight), 1) * farm_factor
    obs += np.random.randn(*obs.shape) * 0.5
    
    return obs

In [ ]:
with open("data/sample_data2.csv", "w") as f:
    f.write("type,latitude,longitude,observation\n")
    
    f.writelines([
        "farm,{},{},1\n".format(x, y)
        for x, y in farm_pixels
    ])
    
    obs = get_observations(wells, farm_pixels, farm_factor, distance_weight, base_water_level)
    f.writelines([
        "well,{},{},{}\n".format(x, y, o)
        for (x, y), o in zip(wells, obs)
    ])

In [ ]:
base_water_level

In [ ]:
distance_weight

In [ ]:
farm_factor